<a href="https://colab.research.google.com/github/amiqdam/learning-journey/blob/main/Product_Expenses_Tracker_with_Multimodal_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mini-Product:

Expenses Tracker with Multimodal AI

In [29]:
# Install dependencies
!pip install -q openai gspread pandas

In [30]:
# Import required library
import os
from openai import OpenAI
import pandas as pd
import base64
import json

# Dedicated in colab environment
import gspread
from google.colab import auth
from google.auth import default
from google.colab import userdata

In [31]:
# Import OpenAI API
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [32]:
# Global Variables
client = OpenAI()
EXPENSE_EXTRACTION_PROMPT = """
You are a data extraction and normalization engine.

Your task is to extract expense information from user input (text, image, or audio)
and output it in STRICT JSON format.

RULES (MANDATORY):
1. Output MUST be valid JSON only.
2. Do NOT add explanations, comments, markdown, or extra text.
3. Do NOT wrap the JSON in code blocks.
4. Use EXACT field names as defined.
5. If a value is missing or not mentioned, infer it if reasonably possible.
   Otherwise, set it to null.
6. Qty and Price Per Item MUST be numbers (not strings).
7. Total Price (Item x Qty) MUST be calculated as Qty × Price Per Item.
8. Date format MUST be YYYY-MM-DD.

OUTPUT FORMAT (SINGLE ITEM):

{
  "Date": "YYYY-MM-DD",
  "Store": "Store name",
  "Item": "Item name",
  "Category": "Item category",
  "Qty": number,
  "Price Per Item": number,
  "Total Price (Item x Qty)": number
}

If multiple items are detected, output an ARRAY of objects
using the same schema.

DO NOT include any fields outside this schema.
"""

In [46]:
# Functions modularity
## Helper
def load_photo(path):
    with open(path, "rb") as f:
        return {
            "type": "input_image",
            "image_url": f"data:image/jpeg;base64,{base64.b64encode(f.read()).decode()}"
        }


def load_audio(path):
    with open(path, "rb") as f:
        return {
            "type": "input_audio",
            "input_audio": {
                "data": base64.b64encode(f.read()).decode(),
                "format": "mp4"
            }
        }


def load_text(text):
    return {
        "type": "input_text",
        "text": text
    }

# Main
def multimodal_ai(prompt=None, photo=None, audio=None):
  """
  Function for multimodal AI

  """
  content = []
  content.append(load_text(EXPENSE_EXTRACTION_PROMPT))

  # Handle audio input by transcribing it first
  if audio:
    try:
      with open(audio, "rb") as audio_file:
        # Use client.audio.transcriptions.create for Speech-to-Text
        transcript_response = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
      # Add the transcribed text to the content list
      content.append(load_text(f"Audio Transcript: {transcript_response.text}"))
      print(f"Audio transcribed: {transcript_response.text[:200]}...")
    except Exception as e:
      print(f"Error transcribing audio: {e}")

  if prompt:
    content.append(load_text(prompt))

  if photo:
    content.append(load_photo(photo))

  if not content:
    raise ValueError("No content provided")

  response = client.responses.create(
      model="gpt-4.1",
      input=[
          {
              "role": "user",
              "content": content
          }
      ]
    )
  parsed = json.loads(response.output_text)
  return parsed

# Google Sheets
def sheets_opener():
  """
  Function for updating sheets
  """
  auth.authenticate_user()

  creds, _ = default()
  gc = gspread.authorize(creds)

  spreadsheet = gc.open("Expenses Tracker")
  return spreadsheet.worksheet("Expenses")

def update_sheets(sheet, expense_data):
    headers = [
        "Date",
        "Store",
        "Item",
        "Category",
        "Qty",
        "Price Per Item",
        "Total Price (Item x Qty)"
    ]

    # Handle single object or list
    if isinstance(expense_data, dict):
        expense_data = [expense_data]

    for item in expense_data:
        row = [item.get(h) for h in headers]
        sheet.append_row(row)

In [34]:
# Open the sheets connector and create First row
sheet = sheets_opener()

sheet.update(range_name='A1:G1', values=[['Date', 'Store', 'Item', 'Category', 'Qty', 'Price Per Item', 'Total Price (Item x Qty)']])

{'spreadsheetId': '11Im2fZxNnkL_3InbyjejzM22hqxRa-fg5RaFCexjteg',
 'updatedRange': 'Expenses!A1:G1',
 'updatedRows': 1,
 'updatedColumns': 7,
 'updatedCells': 7}

In [37]:
# Give the input to update it to the sheets (test for the text)
prompt = "Tadi beli 20 susu Indomilk di Indomaret, satunya harganya 18 ribu, tanggal 12 Januari 2026"
result = multimodal_ai(prompt)
update_sheets(sheet, result)

In [ ]:
# Example of Receipt
!wget https://blog.rosihanari.net/wp-content/uploads/2022/05/WhatsApp-Image-2022-05-22-at-7.30.08-PM.jpeg

In [39]:
# Give the input to update it to the sheets (test for the image)
photo_path = "/content/WhatsApp-Image-2022-05-22-at-7.30.08-PM.jpeg"
result = multimodal_ai(photo=photo_path)
update_sheets(sheet, result)

In [49]:
# Give the input to update it to the sheets (test for the voice)
audio_path="/content/WhatsApp Audio 2026-01-13 at 16.00.49.mp4"
result = multimodal_ai(
    prompt="Extract expense data from this audio in Bahasa Indonesia",
    audio=audio_path
)
update_sheets(sheet, result)

Audio transcribed: Tanggal 2 Januari 2026, saya membeli 50 bungkus rokok Marlboro. Satu bungkusnya harganya Rp50.000 di Tokok Lontong Akiyong....
